# Installation Notebook for Ubuntu Desktop 22.04

# Installation:

```shell
sudo apt-get install -y git python3.10-venv

cd ~/git/repos

git clone https://github.com/michimussato/OpenStudioLandscapes-Installer.git

cd OpenStudioLandscapes-Installer/ubuntu/22.04

python3.10 -m venv .venv
source .venv/bin/activate

pip install --upgrade pip setuptools setuptools_scm wheel

pip install notebook
```

```shell
jupyter notebook
```

## Step 1

Constants

In [ ]:
SHELL_SCRIPTS_PREFIX: str = "ubuntu_desktop_2204__"
URL_HARBOR: str = "http://harbor.farm.evil:80"
ADMIN_HARBOR: str = "admin"
PASSWORD_HARBOR: str = "Harbor12345"

## Step 2

Set `OPENSTUDIOLANDSCAPES` variable so that the installer knows where to pull the source to.

In [ ]:
import pathlib
from getpass import getuser


OPENSTUDIOLANDSCAPES_DIR = None

while OPENSTUDIOLANDSCAPES_DIR is None:
    default_openstudiolandscapes_base = "~/git/repos"
    default_openstudiolandscapes_subdir = "OpenStudioLandscapes"

    openstudiolandscapes_base = pathlib.Path(input(f"Install base dir ({default_openstudiolandscapes_base}): ".strip()) or default_openstudiolandscapes_base).expanduser()

    if not openstudiolandscapes_base.is_absolute():
        print(f"ERROR: Directory {openstudiolandscapes_base.as_posix()} is not absolute (~ is allowed).")
        continue
    if not openstudiolandscapes_base.exists():
        print(f"ERROR: Directory {openstudiolandscapes_base.as_posix()} does not exist. Create it first.")
        continue
    if openstudiolandscapes_base.is_file():
        print(f"ERROR: Install Directory {openstudiolandscapes_base.as_posix()} is a file. Cannot continue.")
        continue

    try:
        probe = pathlib.Path(openstudiolandscapes_base / ".openstudiolandscapes_probe")
        probe.mkdir(parents=True, exist_ok=True)
        probe.rmdir()
        del probe
    except Exception as e:
        print(f"ERROR: Unable to write to {openstudiolandscapes_base.as_posix()}: {e}")
        print(f"Make sure we have write permissions to `{openstudiolandscapes_base.as_posix()}` so that we can create a subdirectory.")
        print(f"i.e `sudo chown -R {getuser()}: {openstudiolandscapes_base.as_posix()}`.")
        continue

    openstudiolandscapes_subdir = input(f"Install sub dir ({default_openstudiolandscapes_subdir}): ".strip()) or default_openstudiolandscapes_subdir
    # Todo:
    #  - [ ] Maybe some more checks here
    OPENSTUDIOLANDSCAPES_DIR = pathlib.Path(openstudiolandscapes_base, openstudiolandscapes_subdir).expanduser()

## Step 2

Source basic functions:
- `script_run()`
- `_get_terminal_size()`
- `sudo_pass()`

In [ ]:
import shutil
import shlex
import subprocess
from getpass import getpass
from typing import Tuple


def script_run(
    sudo: bool = False,
    *,
    script: pathlib.Path,
) -> Tuple[bytes, bytes]:

    cmd = [
        shutil.which("bash"),
        script.as_posix(),
    ]

    if sudo:
        cmd.insert(0, shutil.which("sudo"))
        cmd.insert(1, "--stdin")
        cmd.insert(2, "--reset-timestamp")

    with open(script.as_posix(), "r") as f:
        lines = f.readlines()
        print(" COMMAND ".center(_get_terminal_size()[0], "-"))
        print(f" {shlex.join(cmd)} ".center(_get_terminal_size()[0], " "))
        print(" SCRIPT START ".center(_get_terminal_size()[0], "-"))
        lno = 0
        len_ = len(str(len(lines)))
        for l in lines:
            lno += 1
            print(f"{str(lno).ljust(len_)}: {l.rstrip()}")
        print(" SCRIPT END ".center(_get_terminal_size()[0], "-"))

    try:
        proc = subprocess.run(
            cmd,
            input=None if not sudo else sudo_pass(),
            check=True,
            # cwd=script_prep.parent.as_posix(),
            # env=os.environ,
        )
    except subprocess.CalledProcessError as e:
        print(cmd)
        # with open(script.as_posix(), "r") as f:
        #     print(f.read())
        raise e

    if proc.returncode:
        raise RuntimeError(proc)

    return proc.stdout, proc.stderr


def _get_terminal_size() -> Tuple[int, int]:
    # https://stackoverflow.com/a/14422538
    # https://stackoverflow.com/a/18243550
    cols, rows = shutil.get_terminal_size((80, 20))
    return cols, rows


def sudo_pass() -> bytes:
    # Todo:
    #  - [ ] Mechanism to verify that sudo password is correct
    #  - [ ] implement asterisks
    print(" ENTER PASSWORD ".center(_get_terminal_size()[0], "="))
    _sudo_pass = getpass(prompt=f"Sudo Password for User `{getuser()}`: ")
    return _sudo_pass.encode()

## Step 3 - Disable Unattended Upgrades Unit

Disable Unattended Upgrades `systemd` unit.
`sudo`: yes

### Source Write Script Function

In [ ]:
import pathlib
import tempfile


def script_disable_unattended_upgrades() -> pathlib.Path:
    print(" DISABLE UNATTENDED UPGRADES ".center(_get_terminal_size()[0], "#"))
    with tempfile.NamedTemporaryFile(
            delete=False,
            encoding="utf-8",
            prefix="ubuntu_2204_",
            suffix=".sh",
            mode="x",
    ) as script:
        script.writelines(
            [
                "#!/bin/env bash\n",
                "\n",
                "\n",
                "sudo systemctl disable --now unattended-upgrades\n",
            ]
        )

        script.writelines(
            [
                "\n",
                "exit 0\n",
            ]
        )

        return pathlib.Path(script.name)

### Run Script

In [ ]:
ret_script_disable_unattended_upgrades = script_run(
    sudo=True,
    script=script_disable_unattended_upgrades(),
)

## Step 4 - Prepare System

`sudo`: yes

### Source Write Script Function

In [ ]:
import pathlib
import tempfile


def script_prep() -> pathlib.Path:
    print(" PREP ".center(_get_terminal_size()[0], "#"))
    with tempfile.NamedTemporaryFile(
            delete=False,
            encoding="utf-8",
            prefix=SHELL_SCRIPTS_PREFIX,
            suffix=".sh",
            mode="x",
    ) as script:

        prep_pkgs = [
            "openssh-server",
            "git",
            "htop",
            "vim",
            "graphviz",
        ]
        script.writelines(
            [
                "#!/bin/env bash\n",
                "\n",
                "\n",
                "sudo apt-get update\n",
                "sudo apt-get upgrade -y\n",
                "\n",
                f"sudo apt-get install --no-install-recommends -y {' '.join(prep_pkgs)}\n",
                "\n",
                "sudo apt-get -y autoremove\n",
                "sudo apt-get clean\n",
                "\n",
                "sudo systemctl enable --now ssh\n",
            ]
        )

        script.writelines(
            [
                "\n",
                "exit 0\n",
            ]
        )

        return pathlib.Path(script.name)

### Run Script

In [ ]:
ret_script_prep = script_run(
    sudo=True,
    script=script_prep(),
)

## Step 5 - Clone OpenStudioLandscapes Repository

`sudo`: no

### Source Write Script Function

In [ ]:
import pathlib
import tempfile


def script_clone_openstudiolandscapes(
    openstudiolandscapes_repo_dir: pathlib.Path,
    ssh_key_file: pathlib.Path = pathlib.Path("~/.ssh/id_ed25519").expanduser(),
    known_hosts_file: pathlib.Path = pathlib.Path("~/.ssh/known_hosts").expanduser(),
) -> pathlib.Path:

    # Todo
    #  - [ ] We *could* add some checks here in case git clone fails.
    #        However, ssh authentication is only necessary while repo
    #        private.
    #  - [ ] git clone fails silently if ~/git/repos/OpenStudioLandscapes
    #        already exists. FIX!

    print(" CLONE OPENSTUDIOLANDSCAPES ".center(_get_terminal_size()[0], "#"))

    print(" ENTER EMAIL ".center(_get_terminal_size()[0], "="))
    email = input("Enter your email: ")

    with tempfile.NamedTemporaryFile(
            delete=False,
            encoding="utf-8",
            prefix=SHELL_SCRIPTS_PREFIX,
            suffix=".sh",
            mode="x",
    ) as script:
        script.writelines(
            [
                "#!/bin/env bash\n",
                "\n",
                "\n",
                # f"{shutil.which('ssh-keygen')}\n",
                # <<< n means don't overwrite
                f"ssh-keygen -f {ssh_key_file.as_posix()} -N '' -t ed25519 -C \"{email}\" <<< n\n",
                "eval \"$(ssh-agent -s)\"\n",
                f"ssh-add {ssh_key_file.as_posix()}\n",
                "\n",
                "echo \"Copy/Paste the following Public Key to GitHub:\"\n",
                "echo \"https://github.com/settings/ssh/new\"\n",
                f"cat {ssh_key_file.as_posix()}.pub\n",
                "\n",
                "while [[ \"$choice_ssh\" != [Yy]* ]]; do\n",
                "    read -r -e -p \"Type [Yy]es when ready... \" choice_ssh\n",
                "done\n",
                "\n",
                f"ssh-keyscan github.com >> {known_hosts_file.as_posix()}\n",
            ]
        )

        # If openstudiolandscapes_repo_dir already exists, we move it out
        # of the way. At least until there is a more finegrained solution
        # in place to deal with existing installations.
        script.writelines(
            [
                "\n",
                f"if [ -d {openstudiolandscapes_repo_dir.as_posix()} ]; then\n",
                "    echo \"Backing up previous Installation...\"\n",
                f"    mv {openstudiolandscapes_repo_dir.as_posix()} {openstudiolandscapes_repo_dir.as_posix()}_$(date +\"%Y-%m-%d_%H-%m-%S\")\n",
                "fi\n",
            ]
        )

        script.writelines(
            [
                "\n",
                f"if [ ! -d {openstudiolandscapes_repo_dir.as_posix()} ]; then\n",
                f"    mkdir -p {openstudiolandscapes_repo_dir.as_posix()}\n",
                "fi\n",
                f"git -C {openstudiolandscapes_repo_dir.parent.as_posix()} clone --tags git@github.com:michimussato/OpenStudioLandscapes.git\n",
            ]
        )

        script.writelines(
            [
                "\n",
                "exit 0\n",
            ]
        )

        return pathlib.Path(script.name)

### Run Script

In [ ]:
ret_script_clone_openstudiolandscapes = script_run(
    sudo=False,
    script=script_clone_openstudiolandscapes(
        openstudiolandscapes_repo_dir=OPENSTUDIOLANDSCAPES_DIR,
    ),
)